In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit, broadcast, count, avg
spark = SparkSession.builder.appName("Jupyter").getOrCreate()



24/12/15 21:44:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [24]:
%%sql 
    
DROP TABLE IF EXISTS bootcamp.MatchesDfBucketed

++
||
++
++

In [25]:
%%sql
DROP TABLE IF EXISTS  bootcamp.MedalMatchesPlayersDfBucketed;

++
||
++
++

In [26]:
%%sql

DROP TABLE IF EXISTS bootcamp.MatchDetailsDfBucketed;

++
||
++
++

In [27]:
%%sql

DROP TABLE IF EXISTS bootcamp.full_annotated_matches_unsorted

++
||
++
++

In [28]:
%%sql

DROP TABLE IF EXISTS bootcamp.full_annotated_matches_sorted

++
||
++
++

In [7]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [8]:
medals_df = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv");
#.withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))


maps_df = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv").select(col("mapid"),col("description").alias("maps_description"), col("name").alias("maps_name"))#.withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))


match_details_df = spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")#.withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))


matches_df = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")#.withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))


medal_matches_players_df = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")#.withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))


In [9]:
match_details_df.write\
    .bucketBy(16, 'match_id') \
    .saveAsTable('bootcamp.MatchDetailsDfBucketed', format='parquet')

matches_df.write\
    .bucketBy(16, 'match_id') \
    .saveAsTable('bootcamp.MatchesDfBucketed', format='parquet')

medal_matches_players_df.write\
    .bucketBy(16, 'match_id') \
    .saveAsTable('bootcamp.MedalMatchesPlayersDfBucketed', format='parquet')

match_details_df_bucketed = spark.table('bootcamp.MatchDetailsDfBucketed')
matches_df_bucketed = spark.table('bootcamp.MatchesDfBucketed')
medal_matches_players_df_bucketed = spark.table('bootcamp.MedalMatchesPlayersDfBucketed')

24/12/15 21:44:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
#matches_df.join(match_details_df, 'match_id').explain()

In [11]:
bucket_joined = matches_df_bucketed.join( 
    match_details_df_bucketed, 'match_id', 'left').join(
    medal_matches_players_df_bucketed, ['match_id', 'player_gamertag'], 'left')
#
bucket_joined.show()

+--------------------+---------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+----------+-----+
|            match_id|player_gamertag|    

In [12]:
result_df1 = bucket_joined.join(broadcast(medals_df), "medal_id")

full_annotated_matches = result_df1.join(broadcast(maps_df), "mapid")
full_annotated_matches.show()

+--------------------+----------+--------------------+---------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+-----+--------------------+-----------+----------

In [13]:
full_annotated_matches.write.mode("overwrite").saveAsTable("bootcamp.full_annotated_matches_unsorted")

In [14]:
full_annotated_matches.groupby(
    "player_gamertag").agg(avg("player_total_kills").alias('mtc_cnt')).orderBy(col('mtc_cnt').desc()).show(1)
#).show()#write.bucketBy(16,"match_id").mode('overwrite').saveAsTable("bootcamp.match_details_bucketed")

+---------------+-------+
|player_gamertag|mtc_cnt|
+---------------+-------+
|   gimpinator14|  109.0|
+---------------+-------+
only showing top 1 row



In [16]:
#full_annotated_matches.createOrReplaceTempView("EMP")

# PySpark SQL
sql_str="select playlist_id, count(distinct match_id) as mtc_cnt" \
" from bootcamp.full_annotated_matches_unsorted "  \
" group by playlist_id " \
"order by count(distinct match_id) desc"
spark.sql(sql_str).show(1)

+--------------------+-------+
|         playlist_id|mtc_cnt|
+--------------------+-------+
|f72e0ef0-7c4a-430...|   7640|
+--------------------+-------+
only showing top 1 row



In [17]:
#full_annotated_matches.createOrReplaceTempView("EMP")

# PySpark SQL
sql_str="select mapid, maps_name, count(distinct match_id) as mtc_cnt" \
" from bootcamp.full_annotated_matches_unsorted "  \
" group by mapid, maps_name " \
"order by count(distinct match_id) desc"
spark.sql(sql_str).show(1)

+--------------------+--------------+-------+
|               mapid|     maps_name|mtc_cnt|
+--------------------+--------------+-------+
|c7edbf0f-f206-11e...|Breakout Arena|   7032|
+--------------------+--------------+-------+
only showing top 1 row



In [18]:
sql_str="select mapid, maps_name, count(match_id) as medal_cnt" \
" from bootcamp.full_annotated_matches_unsorted "  \
'where classification = "KillingSpree"'\
" group by mapid, maps_name " \
"order by count(distinct match_id) desc"
spark.sql(sql_str).show(1)



+--------------------+--------------+---------+
|               mapid|     maps_name|medal_cnt|
+--------------------+--------------+---------+
|c7edbf0f-f206-11e...|Breakout Arena|     6734|
+--------------------+--------------+---------+
only showing top 1 row



In [21]:
full_annotated_matches

first_sort_df = full_annotated_matches.sortWithinPartitions(col("mapid"),col("playlist_id"),col("match_id"))

#sorted = df.repartition(10, col("event_date")) \
#        .sortWithinPartitions(col("event_date")) \
#        .withColumn("event_time", col("event_time").cast("timestamp")) \

first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.full_annotated_matches_sorted")

In [23]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM bootcamp.full_annotated_matches_unsorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM bootcamp.full_annotated_matches_sorted.files





size,num_files,unsorted
23730402,13,unsorted
22375541,13,sorted


In [ ]:
%%sql

DROP TABLE IF EXISTS bootcamp.match_details_bucketed

In [ ]:
  - Disabled automatic broadcast join with `spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")`
  - Explicitly broadcast JOINs `medals` and `maps`
  - Bucket join `match_details`, `matches`, and `medal_matches_players` on `match_id` with `16` buckets
  - Aggregate the joined data frame to figure out questions like:
    - Which player averages the most kills per game?
    - Which playlist gets played the most?
    - Which map gets played the most?
    - Which map do players get the most Killing Spree medals on?
  - With the aggregated data set
    - Try different `.sortWithinPartitions` to see which has the smallest data size (hint: playlists and maps are both very low cardinality)